<center> 
     <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300"> 
</center>

# Assignment: Notebook for Peer Assignment
Estimated time needed: 45 minutes


# Assignment Scenario

Congratulations! You have just been hired by a US Venture Capital firm as a data analyst.

The company is considering foreign grain markets to help meet its supply chain requirements for its recent investments in the microbrewery and microdistillery industry, which is involved with the production and distribution of craft beers and spirits.

Your first task is to provide a high level analysis of crop production in Canada. Your stakeholders want to understand the current and historical performance of certain crop types in terms of supply and price volatility. For now they are mainly interested in a macro-view of Canada's crop farming industry, and how it relates to the relative value of the Canadian and US dollars.


# Introduction

Using this R notebook you will:

1.  Understand four datasets 
2.  Load the datasets into four separate tables in a Db2 database
3.  Execute SQL queries unsing the RODBC R package to answer assignment questions 

You have already encountered two of these datasets in the previous practice lab. You will be able to reuse much of the work you did there to prepare your database tables for executing SQL queries.


# Understand the datasets

To complete the assignment problems in this notebook you will be using subsetted snapshots of two datasets from Statistics Canada, and one from the Bank of Canada. The links to the prepared datasets are provided in the next section; the interested student can explore the landing pages for the source datasets as follows:

1.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01&pid=3210035901">Canadian Principal Crops (Data & Metadata)</a>
2.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01&pid=3210007701">Farm product prices (Data & Metadata)</a>
3.  <a href="https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01">Bank of Canada daily average exchange rates</a>


### 1. Canadian Principal Crops Data *

This dataset contains agricultural production measures for the principle crops grown in Canada, including a breakdown by province and teritory, for each year from 1908 to 2020.

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A detailed description of this dataset can be obtained from the StatsCan Data Portal at:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210035901  
Detailed information is included in the metadata file and as header text in the data file, which can be downloaded - look for the 'download options' link.  

### 2. Farm product prices

This dataset contains monthly average farm product prices for Canadian crops and livestock by province and teritory, from 1980 to 2020 (or 'last year', whichever is greatest).

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A description of this dataset can be obtained from the StatsCan Data Portal at:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210007701 
The information is included in the metadata file, which can be downloaded - look for the 'download options' link.  

### 3. Bank of Canada daily average exchange rates *

This dataset contains the daily average exchange rates for multiple foreign currencies. Exchange rates are expressed as 1 unit of the foreign currency converted into Canadian dollars. It includes only the latest four years of data, and the rates are published once each business day by 16:30 ET.

For this assignment you will use a snapshot of this dataset with only the USD-CAD exchange rates included (see next section). We have also prepared a monthly averaged version which you will be using below.

A brief description of this dataset and the original dataset can be obtained from the Bank of Canada Data Portal at:
https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates/

( * these datasets are the same as the ones you used in the practice lab)


### Dataset URLs

  1.  Annual Crop Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv 

  2.  Farm product prices: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv
  
  3.  Daily FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv
  
  4.  Monthly FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv
  

<span style="color:red">**IMPORTANT:**</span> You will be loading these datasets directly into R data frames from these URLs instead of from the StatsCan and Bank of Canada portals. The versions provided at these URLs are simplified and subsetted versions of the original datasets.


#### Now let's load these datasets into four separate Db2 tables.
Let's first load the RODBC package:


In [11]:
library(RODBC);

## Problem 1
#### Create tables
Establish a connection to the Db2 database, and create the following four tables using the RODBC package in R. 
Use the separate cells provided below to create each of your tables.

1.  **CROP_DATA**
2.  **FARM_PRICES**
3.  **DAILY_FX**
4.  **MONTHLY_FX**  

The previous practice lab will help you accomplish this.


### Solution 1


In [12]:
# Establish database connection
dsn_driver <- "{IBM DB2 ODBC Driver}"
dsn_database <- "bludb"            
dsn_hostname <- "9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_port <- "32459"  
dsn_protocol <- "TCPIP"            
dsn_uid <- "phg37060"        
dsn_pwd <- "A78tNpyWBdk7HmSn"      
dsn_security <- "ssl"
conn_path <- paste("DRIVER=",dsn_driver,
                  ";DATABASE=",dsn_database,
                  ";HOSTNAME=",dsn_hostname,
                  ";PORT=",dsn_port,
                  ";PROTOCOL=",dsn_protocol,
                  ";UID=",dsn_uid,
                  ";PWD=",dsn_pwd,
                  ";SECURITY=",dsn_security,        
                    sep="")
conn <- odbcDriverConnect(conn_path)
conn

RODBC Connection 1
Details:
  case=nochange
  DRIVER={IBM DB2 ODBC DRIVER}
  UID=phg37060
  PWD=******
  DATABASE=bludb
  HOSTNAME=9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud
  PORT=32459
  PROTOCOL=TCPIP
  SECURITY=SSL

In [13]:
sql.info <- sqlTypeInfo(conn)
conn.info <- odbcGetInfo(conn)
conn.info["DBMS_Name"]
conn.info["DBMS_Ver"]
conn.info["Driver_ODBC_Ver"]

DBMS_Name 
"DB2/LINUXX8664"

DBMS_Ver 
"11.05.0800"

Driver_ODBC_Ver 
        "03.51"

In [14]:
myschema <- "PHG37060" # e.g. "ZJH17769"
tables <- c("CROP_DATA", "FARM_PRICES","DAILY_FX","MONTHLY_FX")
    
    for (table in tables){  
      # Drop School table if it already exists
      out <- sqlTables(conn, tableType = "TABLE", schema = myschema, tableName =table)
      if (nrow(out)>0) {
        err <- sqlDrop (conn, paste(myschema,".",table,sep=""), errors=FALSE)  
        if (err==-1){
          cat("An error has occurred.\n")
          err.msg <- odbcGetErrMsg(conn)
          for (error in err.msg) {
            cat(error,"\n")
          }
        } else {
          cat ("Table: ",  myschema,".",table," was dropped\n")
        }
      } else {
          cat ("Table: ",  myschema,".",table," does not exist\n")
      }
    }

Table:  PHG37060 . CROP_DATA  was dropped
Table:  PHG37060 . FARM_PRICES  was dropped
Table:  PHG37060 . DAILY_FX  was dropped
Table:  PHG37060 . MONTHLY_FX  was dropped


In [15]:
# CROP_DATA:
df1 <- sqlQuery(conn, "CREATE TABLE CROP_DATA (
                            CD_ID varchar(26) NOT NULL, 
                            YEAR varchar(75) not null,
                            CROP_TYPE varchar(75) NOT NULL, 
                            GEO VARCHAR(75) NOT NULL, 
                            SEEDED_AREA VARCHAR(50) NOT NULL, 
                            HARVESTED_AREA VARCHAR(50), 
                            PRODUCTION VARCHAR(75) NOT NULL,
                            AVG_YIELD VARCHAR(75) NOT NULL, 
                            PRIMARY KEY (CD_ID))",
errors=FALSE)

In [16]:
if (df1 == -1){
  cat ("An error has occurred.\n")
  msg <- odbcGetErrMsg(conn)
  print (msg)
} else {
  cat ("Table was created successfully.\n")
}

Table was created successfully.


In [17]:
# FARM_PRICES:
df2 <- sqlQuery(conn, "CREATE TABLE FARM_PRICES (
                            CD_ID varchar(26) NOT NULL, 
                            DATE varchar(155) not null,
                            CROP_TYPE char(155) NOT NULL, 
                            GEO VARCHAR(715) NOT NULL, 
                            PRICE_PRERMT VARCHAR(520) NOT NUll,
                            FOREIGN KEY(CD_ID) REFERENCES CROP_DATA(CD_ID) on delete restrict)",
errors=FALSE)


In [18]:
if (df2 == -1){
  cat ("An error has occurred.\n")
  msg <- odbcGetErrMsg(conn)
  print (msg)
} else {
  cat ("Table was created successfully.\n")
}

Table was created successfully.


In [19]:
# DAILY_FX:
df3 <- sqlQuery(conn, "CREATE TABLE DAILY_FX (
                            DFX_ID VARCHAR(6) NOT NULL, 
                            DATE varchar(15) not null, 
                            FXUSDCAD VARCHAR(75) NOT NULL,  
                            PRIMARY KEY (DFX_ID))",
errors=FALSE)



In [20]:
if (df3 == -1){
  cat ("An error has occurred.\n")
  msg <- odbcGetErrMsg(conn)
  print (msg)
} else {
  cat ("Table was created successfully.\n")
}

Table was created successfully.


In [21]:
# MONTHLY_FX:
df4 <- sqlQuery(conn, "CREATE TABLE MONTHLY_FX (
                            DFX_ID varchar(6) NOT NULL, 
                            DATE varchar(15) not null, 
                            FXUSDCAD VARCHAR(75) NOT NULL,  
                            FOREIGN KEY (DFX_ID) references DAILY_FX(DFX_ID)on delete restrict)",
                errors=FALSE)


In [22]:
if (df4 == -1){
  cat ("An error has occurred.\n")
  msg <- odbcGetErrMsg(conn)
  print (msg)
} else {
  cat ("Table was created successfully.\n")
}

Table was created successfully.


## Problem 2
#### Read Datasets and Load Tables
Read the datasets into R dataframes using the urls provided above. Then load your tables.


###  Solution 2


In [23]:
tab.frame <- sqlTables(conn, schema=myschema)
nrow(tab.frame)
tab.frame$TABLE_NAME

[1] 15

[1] "BOARD"       "COUNTRY"     "CROP_DATA"   "DAILY_FX"    "DEPARTMENTS"
 [6] "EMPLOYEES"   "FARM_PRICES" "JOBS"        "JOB_HISTORY" "LOCATIONS"  
[11] "MONTHLY_FX"  "PET"         "PETRESCUE"   "PETSALE"     "SCHOOL"

In [24]:
cropdf <- read.csv("/resources/data/samples/final/Annual_Crop_Data (1).csv" , header = TRUE)

In [25]:
sqlSave(conn, cropdf, "CROP_DATA", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)

In [26]:
farmdf <- read.csv("/resources/data/samples/final/Monthly_Farm_Prices.csv",header= TRUE )

In [ ]:
sqlSave(conn, farmdf, "FARM_PRICES", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE , safer = FALSE)

In [ ]:
daily_fx <- read.csv("/resources/data/samples/final/Daily_FX.csv" , header = TRUE)

In [ ]:
sqlSave(conn, daily_fx, "DAILY_FX", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)

In [ ]:
monthly_fx <- read.csv("/resources/data/samples/final/Monthly_FX.csv",header= TRUE )

In [ ]:
sqlSave(conn, monthly_fx, "MONTHLY_FX", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE,safer=FALSE)

In [ ]:
head(farmdf)

## Now execute SQL queries using the RODBC R package to solve the assignment problems.

## Problem 3
#### How many records are in the farm prices dataset?


### Solution 3


In [ ]:
sqlQuery(conn, "SELECT count(*) as abc from FARM_PRICES;")


## Problem 4
#### Which geographies are included in the farm prices dataset?


### Solution 4


In [184]:
sqlQuery(conn, "SELECT unique(GEO) from FARM_PRICES;")

,GEO
,<fct>
1,Alberta
2,Saskatchewan


## Problem 5
#### How many hectares of Rye were harvested in Canada in 1968?


### Solution 5


In [225]:
sqlQuery(conn, "SELECT HARVESTED_AREA from CROP_DATA where YEAR LIKE '1968%' and GEO = 'Canada' and CROP_TYPE='Rye';")


,HARVESTED_AREA
,<int>
1,274100


## Problem 6
#### Query and display the first 6 rows of the farm prices table for Rye.


### Solution 6


In [202]:
sqlQuery(conn,"SELECT * FROM FARM_PRICES where CROP_TYPE='Rye' LIMIT 6;")

,CD_ID,DATE,CROP_TYPE,GEO,PRICE_PRERMT
,<int>,<fct>,<fct>,<fct>,<dbl>
1,4,1985-01-01,Rye,Alberta,100.77
2,5,1985-01-01,Rye,Saskatchewan,109.75
3,10,1985-02-01,Rye,Alberta,95.05
4,11,1985-02-01,Rye,Saskatchewan,103.46
5,16,1985-03-01,Rye,Alberta,96.77
6,17,1985-03-01,Rye,Saskatchewan,106.38


## Problem 7
#### Which provinces grew Barley? 


### Solution 7


In [203]:
sqlQuery(conn, "SELECT GEO FROM FARM_PRICES where CROP_TYPE='Barley'; ")

,GEO
,<fct>
1,Alberta
2,Saskatchewan
3,Alberta
4,Saskatchewan
5,Alberta
6,Saskatchewan
7,Alberta
8,Saskatchewan
9,Alberta


## Problem 8
#### Find the first and last dates for the farm prices data.


### Solution 8


In [211]:
sqlQuery(conn, "SELECT MAX(DATE) AS MAXIMUN_DATE , MIN(DATE) AS MINIMUM_DATE FROM FARM_PRICES")

,MAXIMUN_DATE,MINIMUM_DATE
,<fct>,<fct>
1,2020-12-01,1985-01-01


## Problem 9
#### Which crops have ever reached a farm price greater than or equal to &#0036;350 per metric tonne?


### Solution 9


In [212]:
sqlQuery(conn, "SELECT CROP_TYPE,PRICE_PRERMT FROM FARM_PRICES where PRICE_PRERMT >= '350';")

,CROP_TYPE,PRICE_PRERMT
,<fct>,<dbl>
1,Canola,350.04
2,Canola,363.84
3,Canola,364.36
4,Canola,354.58
5,Canola,359.99
6,Canola,350.47
7,Canola,354.05
8,Canola,354.82
9,Canola,357.37


## Problem 10
#### Rank the crop types harvested in Saskatchewan in the year 2000 by their average yield. Which crop performed best? 


### Solution 10


In [60]:
sqlQuery(conn,"SELECT CROP_TYPE,GEO,YEAR, RANK() OVER(ORDER BY AVG_YIELD) RANK FROM CROP_DATA where GEO = 'Saskatchewan' and YEAR LIKE '2000%' ;")

,CROP_TYPE,GEO,YEAR,RANK
,<fct>,<fct>,<fct>,<int>
1,Canola,Saskatchewan,2000-12-31,1
2,Rye,Saskatchewan,2000-12-31,2
3,Wheat,Saskatchewan,2000-12-31,3
4,Barley,Saskatchewan,2000-12-31,4


## Problem 11
#### Rank the crops and geographies by their average yield (KG per hectare) since the year 2000. Which crop and province had the highest average yield since the year 2000? 


### Solution 11


In [66]:
sqlQuery(conn, "SELECT CROP_TYPE,GEO ,AVG_YIELD,YEAR, RANK() OVER(ORDER BY AVG_YIELD DESC)RANK FROM CROP_DATA where YEAR LIKE '2000%' ;")

,CROP_TYPE,GEO,AVG_YIELD,YEAR,RANK
,<fct>,<fct>,<int>,<fct>,<int>
1,Barley,Alberta,3000,2000-12-31,1
2,Barley,Canada,3000,2000-12-31,1
3,Barley,Saskatchewan,2800,2000-12-31,3
4,Wheat,Alberta,2500,2000-12-31,4
5,Wheat,Canada,2400,2000-12-31,5
6,Rye,Canada,2300,2000-12-31,6
7,Wheat,Saskatchewan,2200,2000-12-31,7
8,Rye,Alberta,2100,2000-12-31,8
9,Rye,Saskatchewan,2100,2000-12-31,8


## Problem 12
#### Use a subquery to determine how much wheat was harvested in Canada in the most recent year of the data.


### Solution 12


In [73]:
sqlQuery(conn, "SELECT AVG_YIELD,year,CROP_TYPE FROM CROP_DATA where year=(select MAX(YEAR)from CROP_DATA)and GEO ='Canada' and CROP_TYPE = 'Wheat';")

,AVG_YIELD,YEAR,CROP_TYPE
,<int>,<fct>,<fct>
1,3512,2020-12-31,Wheat


## Problem 13
#### Use an implicit inner join to calculate the monthly price per metric tonne of Canola grown in Saskatchewan in both Canadian and US dollars. Display the most recent 6 months of the data.


### Solution 13


In [59]:
sqlQuery (conn, "select PRICE_PRERMT as CAD,PRICE_PRERMT*0.75 as USD from FARM_PRICES F inner join CROP_DATA C on F.CD_ID=C.CD_ID where F.CROP_TYPE='Canola'and F.GEO='Saskatchewan 'LIMIT 6;")

,CAD,USD
,<dbl>,<dbl>
1,339.82,254.8650
2,341.26,255.9450
3,349.77,262.3275
4,364.36,273.2700
5,359.99,269.9925
6,350.47,262.8525


## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                                          |
| ---------- | ------- | ------------- | ----------------------------------------------------------------------------------------------------------- |
| 2021-04-01 | 0.7     | Jeff Grossman | Split Problem 1 solution cell into multiple cells, fixed minor bugs |
| 2021-03-12 | 0.6     | Jeff Grossman | Cleaned up content for production |
| 2021-03-11 | 0.5     | Jeff Grossman | Moved more advanced problems to optional honours module |
| 2021-03-10 | 0.4     | Jeff Grossman | Added introductory and intermediate level problems and removed some advanced problems |
| 2021-03-04 | 0.3     | Jeff Grossman | Moved some problems to a new practice lab as prep for this assignment
| 2021-03-04 | 0.2     | Jeff Grossman | Sorted problems roughly by level of difficulty and relegated more advanced ones to ungraded bonus problems  |
| 2021-02-20 | 0.1     | Jeff Grossman | Started content creation                                                                                    |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
